# Training SimpleNN on CIFAR-10
In this project, you will use the SimpleNN model to perform image classification on CIFAR-10. CIFAR-10 orginally contains 60K images from 10 categories. We split it into 45K/5K/10K images to serve as train/validation/test set. We only release the ground-truth labels of training/validation dataset to you.

## Step 0: Set up the SimpleNN model
As you have practiced to implement simple neural networks in Homework 1, we just prepare the implementation for you.

In [6]:
# import necessary dependencies
import argparse
import os, sys
import time
import datetime
from tqdm import tqdm_notebook as tqdm
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import copy
import torch
torch.cuda.empty_cache()
!pwd

/content


In [7]:
from google.colab import drive
drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/ECE_661/661_Final_Project')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import tools

In [9]:
0# THIS IS WHERE I AM GONNA MAKE MY RESNET - at the start no BN or swish or data augmentation- using RELU to make simple
class Resnet_20(nn.Module):
    def __init__(self):
        super(Resnet_20, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, stride = 1, padding = 1) # I am not sure about this output, should it be 32 or 3? Also coudln't I just use identities by forcing 
        self.conv1_bn = nn.BatchNorm2d(16)

        #starting first section of 6 - feature map 32, channel (features) 16
        self.conv2 = nn.Conv2d(16,16,3, stride = 1, padding = 1)
        self.conv2_bn = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16,16,3, padding = 1)
        self.conv3_bn = nn.BatchNorm2d(16)
        self.conv4 = nn.Conv2d(16,16,3, padding = 1)
        self.conv4_bn = nn.BatchNorm2d(16)
        self.conv5 = nn.Conv2d(16,16,3, padding = 1)
        self.conv5_bn = nn.BatchNorm2d(16)
        self.conv6 = nn.Conv2d(16,16,3, padding = 1)
        self.conv6_bn = nn.BatchNorm2d(16)
        self.conv7 = nn.Conv2d(16,16,3, padding = 1) #it would be here that I need a 1x1 convolution on my bypass that gets fed to lower stuff 
        self.conv7_bn = nn.BatchNorm2d(16)
        self.bp4_conv = nn.Conv2d(16,32,1, stride = 2)
        self.bp4_bn = nn.BatchNorm2d(32) 

        

        #Now starting second section of 6 - now feature map 16, channel (features) 32
        self.conv8 = nn.Conv2d(16,32,3, stride = 2, padding = 1) # HERE COULD BE AN ISSUE - I AM NOT SURE IF I SHOULD PAD OR NOT 
        self.conv8_bn = nn.BatchNorm2d(32) 
        self.conv9 = nn.Conv2d(32,32,3, padding = 1)
        self.conv9_bn = nn.BatchNorm2d(32) 
        self.conv10 = nn.Conv2d(32,32,3, padding = 1)
        self.conv10_bn = nn.BatchNorm2d(32) 
        self.conv11 = nn.Conv2d(32,32,3, padding = 1)
        self.conv11_bn = nn.BatchNorm2d(32) 
        self.conv12 = nn.Conv2d(32,32,3, padding = 1)
        self.conv12_bn = nn.BatchNorm2d(32) 
        self.conv13 = nn.Conv2d(32,32,3, padding = 1) # need 1x1 conv here on bypass that gets fed to lower stuff 
        self.conv13_bn = nn.BatchNorm2d(32)
        self.bp7_conv = nn.Conv2d(32,64,1, stride = 2)
        self.bp7_bn = nn.BatchNorm2d(64) 

        # now starting third section of 6 - feature map is 8, channel (features) 64 (8x8x64)
        self.conv14 = nn.Conv2d(32,64,3, stride = 2, padding = 1) # SAME THING MAYBE I NEED TO PADD MAYBE I DON'T NEED TO 
        self.conv14_bn = nn.BatchNorm2d(64) 
        self.conv15 = nn.Conv2d(64,64,3, stride = 1, padding = 1)
        self.conv15_bn = nn.BatchNorm2d(64) 
        self.conv16 = nn.Conv2d(64,64,3, stride = 1, padding = 1)
        self.conv16_bn = nn.BatchNorm2d(64) 
        self.conv17 = nn.Conv2d(64,64,3, stride = 1, padding = 1)
        self.conv17_bn = nn.BatchNorm2d(64) 
        self.conv18 = nn.Conv2d(64,64,3, stride = 1, padding = 1)
        self.conv18_bn = nn.BatchNorm2d(64) 
        self.conv19 = nn.Conv2d(64,64,3, stride = 1, padding = 1)
        self.conv19_bn = nn.BatchNorm2d(64)  
       

        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1   = nn.Linear(64,10) 

    # THIS IS WHERE I need to fit it all together and worry about my residual stuff 
    def forward(self, x):
        out = F.relu(self.conv1_bn(self.conv1(x))) # this is okay, goes after conv and before activation functionp

        bp1 = out # SAVE THE BYPASS DATA HERE, MAYBE COPY? Maybe not 
      
        out = F.relu(self.conv2_bn(self.conv2(out)))

        out = self.conv3_bn(self.conv3(out))
  
        # IN here I need to add the first bypass data to the current out, then I feed it to the next guy - need to think a little about when I need to do the convolution on the BP
        out = bp1 + out #This totally works! Very chill and cool of pytorch lol
        out = F.relu(out)
        bp2 = out # maybe copy, maybe not, we'll see 
        out  = F.relu(self.conv4_bn(self.conv4(out)))
        out = self.conv5_bn(self.conv5(out))
        out = bp2 + out
        out = F.relu(out)
        bp3 = out
        out = F.relu(self.conv6_bn(self.conv6(out)))
        out = self.conv7_bn(self.conv7(out))
        out = out + bp3
        out = F.relu(out)
        bp4 = out
        bp4_ready = self.bp4_bn(self.bp4_conv(bp4))
        out = F.relu(self.conv8_bn(self.conv8(out)))
        out = self.conv9_bn(self.conv9(out))
        out = bp4_ready+out
        out = F.relu(out)
        bp5 = out
        out = F.relu(self.conv10_bn(self.conv10(out)))
        out = self.conv11_bn(self.conv11(out))
        out = out + bp5
        out = F.relu(out)
        bp6 = out
        out = F.relu(self.conv12_bn(self.conv12(out)))
        out = self.conv13_bn(self.conv13(out))
        out = out + bp6
        out = F.relu(out)
        bp7 = out
        bp7_ready = self.bp7_bn(self.bp7_conv(out))
        out = F.relu(self.conv14_bn(self.conv14(out)))
        out = self.conv15_bn(self.conv15(out))
        out = bp7_ready+out
        out = F.relu(out)
        bp8 = out
        out = F.relu(self.conv16_bn(self.conv16(out)))
        out = self.conv17_bn(self.conv17(out))
        out = out + bp8
        out = F.relu(out)
        bp9 = out
        out = F.relu(self.conv18_bn(self.conv18(out)))
        out = self.conv19_bn(self.conv19(out))
        out = out + bp9
        out = F.relu(out)
        out = self.avg_pool(out)


        out = out.view(out.size(0), -1) # IF THIS BREAKS THEN CHECK HERE, Might need it might not
        #avg pooling here
        #fully connected here
        out = self.fc1(out)
        return out

### Question (a)
Here is a sanity check to verify the implementation of SimpleNN. 
You need to:
1. Write down your code.
2. **In the PDF report**, give a brief description on how the code helps you know that SimpleNN is implemented correctly.

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device =='cuda':
    print("Run on GPU...")
else:
    print("Run on CPU...")

# Model Definition  
net_res = Resnet_20().to(device)


#Passing one image through - if it works for one it would work as we scale 
data = torch.randn(1,3,32,32)
data = data.to(device)
# Forward pass "data" through "net" to get output "out" 
out = net_res.forward(data)

# Check output shape

#Output shape should be 1x10 - one image, should get 10 scores, one for each possible classification

print(out.detach().cpu().numpy().shape)
assert(out.detach().cpu().numpy().shape == (1,10))
print("Correct output shape - 1x10 for passing in one image")


# Forward pass "data" through "net" to get output "out" 
out = net_res.forward(data)

# Iterate through all the CONV and FC layers of the model
for name, module in net_res.named_modules():
    if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
       
        # Get the weight of the module as a NumPy array
        weight =  module.cpu().weight.detach().numpy()
        # Compute the number of parameters in the weight

        # From slide 17 lecture 4 -- bc weight.shape is [out,in,kernel,kernel]

        # we know num_paramaters would be the product of the shape of our weight

        num_Param = np.product(weight.shape)

       
        
        print(f'{name:10} number parameters: {str(num_Param):10} ')

Run on GPU...
(1, 10)
Correct output shape - 1x10 for passing in one image
conv1      number parameters: 432        
conv2      number parameters: 2304       
conv3      number parameters: 2304       
conv4      number parameters: 2304       
conv5      number parameters: 2304       
conv6      number parameters: 2304       
conv7      number parameters: 2304       
bp4_conv   number parameters: 512        
conv8      number parameters: 4608       
conv9      number parameters: 9216       
conv10     number parameters: 9216       
conv11     number parameters: 9216       
conv12     number parameters: 9216       
conv13     number parameters: 9216       
bp7_conv   number parameters: 2048       
conv14     number parameters: 18432      
conv15     number parameters: 36864      
conv16     number parameters: 36864      
conv17     number parameters: 36864      
conv18     number parameters: 36864      
conv19     number parameters: 36864      
fc1        number parameters: 640        


## Step 1: Set up preprocessing functions
Preprocessing is very important as discussed in the lecture.
You will need to write preprocessing functions with the help of *torchvision.transforms* in this step.
You can find helpful tutorial/API at [here](https://pytorch.org/vision/stable/transforms.html).

### Question (b)
For the question, you need to:
1. Complete the preprocessing code below.
2. **In the PDF report**, briefly describe what preprocessing operations you used and what are the purposes of them.

Hint: 
1. Only two operations are necessary to complete the basic preprocessing here.
2. The raw input read from the dataset will be PIL images.
3. Data augmentation operations are not mendatory, but feel free to incorporate them if you want.
4. Reference value for mean/std of CIFAR-10 images (assuming the pixel values are within [0,1]): mean (RGB-format): (0.4914, 0.4822, 0.4465), std (RGB-format): (0.2023, 0.1994, 0.2010)

In [11]:
# useful libraries
import torchvision
import torchvision.transforms as transforms

#############################################
# your code here
# specify preprocessing function
transform_train = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010)),
        #torchvision.transforms.RandomCrop((32,32), padding = 4),
        #torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.RandomRotation((90,90)) # Play around with number of degrees for random rotation
        ])

transform_val = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))
        ])
#############################################

## Step 2: Set up dataset and dataloader

### Question (c)
Set up the train/val datasets and dataloders that are to be used during the training. Check out the [official API](https://pytorch.org/docs/stable/data.html) for more information about **torch.utils.data.DataLoader**.

Here, you need to:
1. Complete the code below.

In [12]:

# do NOT change these
import tools
from tools.dataset import CIFAR10 
from torch.utils.data import DataLoader

# a few arguments, do NOT change these
DATA_ROOT = "./data"
TRAIN_BATCH_SIZE = 128
VAL_BATCH_SIZE = 100

#############################################
# your code here
# construct dataset
train_set = CIFAR10(
    root=DATA_ROOT, 
    mode='train', 
    download=True,
    transform= transform_train    # your code - this is specifying the way I will transform the data!

)
val_set = CIFAR10(
    root=DATA_ROOT, 
    mode='val', 
    download=True,
    transform= transform_val    # your code
)

# construct dataloader
train_loader = DataLoader(
    train_set, 
    batch_size=256,  # your code - mult of 2, determines num of img use for forward/back pass
    shuffle=True,     # your code - shuffle to help with convergence - not act required
    num_workers=4
)

print(type(train_loader))

val_loader = DataLoader(
    val_set, 
    batch_size=256,  # your code
    shuffle=False,     # your code
    num_workers=4
)
#############################################

0it [00:00, ?it/s]

Extracting ./data/cifar10_trainval_F22.zip to ./data
Files already downloaded and verified
Using downloaded and verified file: ./data/cifar10_trainval_F22.zip
Extracting ./data/cifar10_trainval_F22.zip to ./data
Files already downloaded and verified
<class 'torch.utils.data.dataloader.DataLoader'>


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## Step 3: Instantiate your SimpleNN model and deploy it to GPU devices.
### Question (d)
You may want to deploy your model to GPU device for efficient training. Please assign your model to GPU if possible. If you are training on a machine without GPUs, please deploy your model to CPUs.

Here, you need to:
1. Complete the code below.
2. **In the PDF report**, briefly describe how you verify that your model is indeed deployed on GPU. (Hint: check $\texttt{nvidia-smi}$.)

## Step 4: Set up the loss function and optimizer
Loss function/objective function is used to provide "feedback" for the neural networks. Typically, we use multi-class cross-entropy as the loss function for classification models. As for the optimizer, we will use SGD with momentum. 

### Question (e)
Here, you need to:
1. Set up the cross-entropy loss as the criterion. (Hint: there are implemented functions in **torch.nn**)
2. Specify a SGD optimizer with momentum. (Hint: there are implemented functions in **torch.optim**)

In [13]:
import torch.nn as nn
import torch.optim as optim
net_res = Resnet_20().to(device)
# hyperparameters, do NOT change right now
# initial learning rate
INITIAL_LR = 0.25

# momentum for optimizer
MOMENTUM = 0.9

# L2 regularization strength
REG = 1e-4

#############################################
# your code here
# create loss function
criterion = torch.nn.CrossEntropyLoss().to(device) # - Should have inputs of predeictions and actual 

# Add optimizer
optimizer = torch.optim.SGD(net_res.parameters(), momentum = MOMENTUM, lr = INITIAL_LR, weight_decay = REG) # - missing paramenters to optimize over, need to pass it weights? or some of criterion losses?
#trying out a different optimizer, adam, that is supposed to converge faster - also I am lessening my decay to .5 instead of .1

#optimizer = torch.optim.Adam(net_res.parameters(), weight_decay = REG, lr = INITIAL_LR)

#############################################

## Step 5: Start the training process.

### Question (f)/(g)
Congratulations! You have completed all of the previous steps and it is time to train our neural network.

Here you need to:
1. Complete the training codes.
2. Actually perform the training.

Hint: Training a neural network usually repeats the following 4 steps: 

**i) Get a batch of data from the dataloader and copy it to your device (GPU).**

**ii) Do a forward pass to get the outputs from the neural network and compute the loss. Be careful about your inputs to the loss function. Are the inputs required to be the logits or softmax probabilities?)**

**iii) Do a backward pass (back-propagation) to compute gradients of all weights with respect to the loss.**

**iiii) Update the model weights with the optimizer.**

You will also need to compute the accuracy of training/validation samples to track your model's performance over each epoch (the accuracy should be increasing as you train for more and more epochs).


In [16]:
# some hyperparameters
# total number of training epochs
print('This is Resnet_20\n\n')
EPOCHS = 200

# the folder where the trained model is saved
CHECKPOINT_FOLDER = "/"

# start the training/validation process
# the process should take about 5 minutes on a GTX 1070-Ti
# if the code is written efficiently.
best_val_acc = 0
current_learning_rate = INITIAL_LR

print("==> Training starts!")
print("="*50)



for i in range(0, EPOCHS):
    # handle the learning rate scheduler.
    if i == 100  or i == 140 or i == 180:
        current_learning_rate = current_learning_rate * .8
        for param_group in optimizer.param_groups:
            param_group['lr'] = current_learning_rate
        print("Current learning rate has decayed to %f" %current_learning_rate)
    if i == 20:
      current_learning_rate = current_learning_rate * .4
      for param_group in optimizer.param_groups:
            param_group['lr'] = current_learning_rate
      print("Current learning rate has decayed to %f" %current_learning_rate)
    
    #######################
    # your code here
    # switch to train mode
    net_res.train()
    
    
    #######################
    
    print("Epoch %d:" %i)
    # this help you compute the training accuracy
    total_examples = 0
    correct_examples = 0

    train_loss = 0 # track training loss if you want
    
    # Train the model for 1 epoch.
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        ####################################
        # your code here
        # copy inputs to device
        inputs = inputs.to(device)

        targets = targets.to(device)
        
        # compute the output and loss
        outputs = net_res(inputs)
        # if batch_idx == 1:
          # print(outputs.shape)
          # print(outputs[0])
          # print(outputs[1][0])

        # I THINK IF THERE IS AN ERROR IT MAY BE HERE - but IDK why it would be one -- CRITERION HAS BUILT IN SOFTMAX SO SHOULD PASS LOGIT, NOT SOFTMAX PROB
        loss = criterion(outputs, targets)

        train_loss += loss
        
        # zero the gradient
        optimizer.zero_grad()
        
        # backpropagation
        loss.backward()
        
        # apply gradient and update the weights - DO I NEED TO DO SOMETHING MORE HERE? 
        optimizer.step()
        
        # count the number of correctly predicted samples in the current batch
        _, predicted = torch.max(outputs, 1)

        correct_examples += predicted.eq(targets).sum().item()
        total_examples += targets.shape[0]
        ####################################
                
    avg_loss = train_loss / len(train_loader)
    avg_acc = correct_examples / total_examples
    print("Training loss: %.4f, Training accuracy: %.4f" %(avg_loss, avg_acc))

    # Validate on the validation dataset
    #######################
    # your code here
    # switch to eval mode
    net_res.eval()
    
    #######################

    # this help you compute the validation accuracy
    total_examples = 0
    correct_examples = 0
    
    val_loss = 0 # again, track the validation loss if you want

    # disable gradient during validation, which can save GPU memory
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            ####################################
            # your code here
            # copy inputs to device
            inputs = inputs.to(device)
            targets = targets.to(device)

            
            # compute the output and loss
            outputs = net_res(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss
            
            # count the number of correctly predicted samples in the current batch
            _, predicted = torch.max(outputs, 1)

            correct_examples += predicted.eq(targets).sum().item()
            total_examples += targets.shape[0]
            ####################################

    avg_loss = val_loss / len(val_loader)
    avg_acc = correct_examples / total_examples
    print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss, avg_acc))
    
    # save the model checkpoint
    if avg_acc > best_val_acc:
        best_val_acc = avg_acc
        if not os.path.exists(CHECKPOINT_FOLDER):
            os.makedirs(CHECKPOINT_FOLDER)
        print("Saving ...")
        state = {'state_dict': net_res.state_dict(),
                 'epoch': i,
                 'lr': current_learning_rate}
        torch.save(state, os.path.join(CHECKPOINT_FOLDER, 'simplenn.pth'))
        
    print('')

print("="*50)
print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")

This is Resnet_20


==> Training starts!
Epoch 0:


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


torch.Size([256, 10])
tensor([-1.2674,  1.7018,  1.0398, -1.2743,  0.1744, -0.1933,  0.2480,  1.1308,
        -0.7346, -0.5039], device='cuda:0', grad_fn=<SelectBackward0>)
tensor(-0.1401, device='cuda:0', grad_fn=<SelectBackward0>)


KeyboardInterrupt: ignored

In [ ]:
!nvidia-smi

Thu Sep 29 21:57:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    43W / 250W |   1737MiB / 16280MiB |     26%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Bonus: with learning rate decay

The following code can help you adjust the learning rate during training. You need to figure out how to incorporate this code into your training loop.
```python
    if i % DECAY_EPOCHS == 0 and i != 0:
        current_learning_rate = current_learning_rate * DECAY
        for param_group in optimizer.param_groups:
            param_group['lr'] = current_learning_rate
        print("Current learning rate has decayed to %f" %current_learning_rate)
```

In [ ]:
!nvidia-smi